In [1]:
%status

{'status': 'healthy',
 'startTime': 'Sun Dec 10 13:17:44 UTC 2023',
 'dbEngineVersion': '1.2.1.0.R7',
 'role': 'writer',
 'dfeQueryEngine': 'viaQueryHint',
 'gremlin': {'version': 'tinkerpop-3.6.2'},
 'sparql': {'version': 'sparql-1.1'},
 'opencypher': {'version': 'Neptune-9.0.20190305-1.0'},
 'labMode': {'ObjectIndex': 'disabled',
  'ReadWriteConflictDetection': 'enabled'},
 'features': {'SlowQueryLogs': 'disabled',
  'ResultCache': {'status': 'disabled'},
  'IAMAuthentication': 'disabled',
  'Streams': 'disabled',
  'AuditLog': 'disabled'},
 'settings': {'clusterQueryTimeoutInMs': '120000',
  'SlowQueryLogsThreshold': '5000'}}

In [2]:
%seed

Dropdown(description='Source type:', options=('', 'samples', 'custom'), style=DescriptionStyle(description_wid…

Dropdown(description='Data model:', layout=Layout(display='none', visibility='hidden'), options=('', 'property…

Dropdown(description='Language:', layout=Layout(display='none', visibility='hidden'), options=('', 'opencypher…

Dropdown(description='Language:', layout=Layout(display='none', visibility='hidden'), options=('', 'opencypher…

Dropdown(description='Data set:', layout=Layout(display='none', visibility='hidden'), options=(), style=Descri…

...

Dropdown(description='Location:', layout=Layout(display='none', visibility='hidden'), options=('Local', 'S3'),…

FileChooser(path='/home/ec2-user/SageMaker', filename='', title='', show_hidden=False, select_desc='Select', c…

Button(description='Submit', layout=Layout(visibility='hidden'), style=ButtonStyle())

Output()

Output()

In [2]:
%%oc -d code -g country -de dist
MATCH p=(a:airport)-[:route]->(b:airport)<-[:contains]-(c:country)
WHERE a.code IN ['LHR'] AND b.code IN ['AMS','JFK','NRT','PER']
RETURN p
LIMIT 4

### 参考资料
https://github.com/langchain-ai/langchain
https://python.langchain.com/docs/use_cases/graph/neptune_cypher_qa


In [3]:
!pip install langchain

In [4]:
from langchain.graphs import NeptuneGraph

#neptune host
host = "neptunedbcluster-nn9wk5uwpeyr.cluster-cj0qabvcmceu.us-west-2.neptune.amazonaws.com"
port = 8182
use_https = True

graph = NeptuneGraph(host=host, port=port, use_https=use_https)

In [5]:
graph.schema

"\n        Node properties are the following:\n        [{'properties': [{'property': 'names', 'type': 'LIST'}, {'property': 'names', 'type': 'STRING'}, {'property': 'primaryName', 'type': 'LIST'}, {'property': 'primaryName', 'type': 'STRING'}], 'labels': 'LOCATION'}, {'properties': [{'property': 'code', 'type': 'STRING'}, {'property': 'type', 'type': 'STRING'}, {'property': 'desc', 'type': 'STRING'}], 'labels': 'continent'}, {'properties': [{'property': 'code', 'type': 'STRING'}, {'property': 'type', 'type': 'STRING'}, {'property': 'desc', 'type': 'STRING'}], 'labels': 'country'}, {'properties': [{'property': 'names', 'type': 'STRING'}, {'property': 'primaryName', 'type': 'STRING'}, {'property': 'names', 'type': 'LIST'}], 'labels': 'SHELF_OFFERING'}, {'properties': [{'property': 'names', 'type': 'STRING'}, {'property': 'primaryName', 'type': 'LIST'}, {'property': 'primaryName', 'type': 'STRING'}, {'property': 'names', 'type': 'LIST'}], 'labels': 'PERSON'}, {'properties': [{'property': 

In [6]:
import sys
import os
os.environ["OPENAI_API_KEY"]="YOURKEY"


In [7]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [8]:
from langchain.chains import NeptuneOpenCypherQAChain
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0, model="gpt-4")
chain = NeptuneOpenCypherQAChain.from_llm(llm=llm, graph=graph, verbose=True, top_K=10,
                                         return_intermediate_steps=True, return_direct=False)
#chain = NeptuneOpenCypherQAChain.from_llm(llm=llm, graph=graph)

PE words:
Generate the query in openCypher format and follow these rules:
Use only the provided relationship types and properties in the schema.
Do not include any explanations or apologies in your responses.


In [12]:
chain.run('''What is the dist from London Heathrow to Perth nonstop?''')



> Entering new NeptuneOpenCypherQAChain chain...
Generated Cypher:
MATCH (a:airport {primaryName: "London Heathrow"})-[r:route]->(b:airport {primaryName: "Perth"}) RETURN r.dist

Full Context:
{'ResponseMetadata': {'HTTPStatusCode': 200, 'HTTPHeaders': {'transfer-encoding': 'chunked', 'content-type': 'application/json;charset=UTF-8'}, 'RetryAttempts': 0}, 'results': []}

> Finished chain.


"I'm sorry, but I don't have the information about the distance from London Heathrow to Perth nonstop."

In [10]:
chain.run('''Can I fly from Austin to Sydney with no stops?''')



> Entering new NeptuneOpenCypherQAChain chain...
Generated Cypher:
MATCH (a:airport {city: 'Austin'}), (b:airport {city: 'Sydney'})
RETURN EXISTS((a)-[:route]->(b)) AS direct_flight_exists

Full Context:
{'ResponseMetadata': {'HTTPStatusCode': 200, 'HTTPHeaders': {'transfer-encoding': 'chunked', 'content-type': 'application/json;charset=UTF-8'}, 'RetryAttempts': 0}, 'results': [{'direct_flight_exists': False}, {'direct_flight_exists': False}, {'direct_flight_exists': False}]}

> Finished chain.


'No, there are no direct flights available from Austin to Sydney.'

In [13]:
chain.run('''How many outgoing routes does the Austin (AUS) airport have?''')



> Entering new NeptuneOpenCypherQAChain chain...
Generated Cypher:
MATCH (a:airport {code: 'AUS'})-[r:route]->() RETURN COUNT(r)

Full Context:
{'ResponseMetadata': {'HTTPStatusCode': 200, 'HTTPHeaders': {'transfer-encoding': 'chunked', 'content-type': 'application/json;charset=UTF-8'}, 'RetryAttempts': 0}, 'results': [{'COUNT(r)': 93}]}

> Finished chain.


'The Austin (AUS) airport has 93 outgoing routes.'

In [14]:
chain.run('''Find the top 10 airports sorted by the number of outgoing routes?
Return just the airport code and the number of routes''')



> Entering new NeptuneOpenCypherQAChain chain...
Generated Cypher:
MATCH (a:airport)-[r:route]->()
WITH a.code AS AirportCode, COUNT(r) AS NumberOfRoutes
RETURN AirportCode, NumberOfRoutes
ORDER BY NumberOfRoutes DESC
LIMIT 10

Full Context:
{'ResponseMetadata': {'HTTPStatusCode': 200, 'HTTPHeaders': {'transfer-encoding': 'chunked', 'content-type': 'application/json;charset=UTF-8'}, 'RetryAttempts': 0}, 'results': [{'AirportCode': 'IST', 'NumberOfRoutes': 307}, {'AirportCode': 'FRA', 'NumberOfRoutes': 307}, {'AirportCode': 'CDG', 'NumberOfRoutes': 293}, {'AirportCode': 'AMS', 'NumberOfRoutes': 282}, {'AirportCode': 'MUC', 'NumberOfRoutes': 270}, {'AirportCode': 'ORD', 'NumberOfRoutes': 264}, {'AirportCode': 'DFW', 'NumberOfRoutes': 251}, {'AirportCode': 'PEK', 'NumberOfRoutes': 248}, {'AirportCode': 'DXB', 'NumberOfRoutes': 247}, {'AirportCode': 'ATL', 'NumberOfRoutes': 242}]}

> Finished chain.


'The top 10 airports sorted by the number of outgoing routes are:\n\n1. IST with 307 routes\n2. FRA with 307 routes\n3. CDG with 293 routes\n4. AMS with 282 routes\n5. MUC with 270 routes\n6. ORD with 264 routes\n7. DFW with 251 routes\n8. PEK with 248 routes\n9. DXB with 247 routes\n10. ATL with 242 routes'